In [54]:
import polars as pl
features_df = pl.scan_parquet("/home/muthu/Desktop/New Folder/notebook/features.parquet"); features_df

In [55]:
X = features_df.drop("muid").collect().to_pandas(); X

,event_count,purchase_count,add_to_cart_count,viewcontent_count,scroll_count,search_count,pageview_count,desktop_count,android_count,ios_count,browser_count,app_count,seo_count,social_count,paid_count,direct_count,session_count,unique_products,unique_categories,unique_locations
0,60,1.0,5.0,13.0,21.0,5.0,15.0,0.0,60.0,0.0,0.0,60.0,0.0,60.0,0.0,0.0,2,35,8,1
1,70,5.0,9.0,20.0,21.0,3.0,12.0,0.0,70.0,0.0,70.0,0.0,0.0,0.0,0.0,70.0,1,47,8,1
2,220,11.0,30.0,61.0,60.0,8.0,50.0,0.0,220.0,0.0,220.0,0.0,0.0,0.0,0.0,220.0,2,149,10,1
3,60,4.0,8.0,11.0,12.0,6.0,19.0,60.0,0.0,0.0,0.0,60.0,0.0,60.0,0.0,0.0,1,42,10,1
4,97,4.0,13.0,25.0,26.0,3.0,26.0,0.0,97.0,0.0,0.0,97.0,0.0,0.0,0.0,97.0,1,69,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,248,18.0,21.0,60.0,77.0,9.0,63.0,0.0,248.0,0.0,0.0,248.0,0.0,0.0,0.0,248.0,14,158,8,1
99996,97,9.0,6.0,26.0,28.0,5.0,23.0,0.0,0.0,97.0,0.0,97.0,0.0,0.0,0.0,97.0,2,65,9,1
99997,52,2.0,5.0,13.0,20.0,0.0,12.0,0.0,0.0,52.0,52.0,0.0,52.0,0.0,0.0,0.0,2,32,7,1
99998,127,5.0,15.0,33.0,37.0,7.0,30.0,127.0,0.0,0.0,0.0,127.0,0.0,0.0,0.0,127.0,1,84,9,1


In [56]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

scaler = StandardScaler()
scaler.fit(X)
scaled_X = scaler.transform(X); scaled_X

array([[-0.57757213, -0.85343026, -0.65070661, ..., -0.6476582 ,
         0.33306149,  0.        ],
       [-0.49530709, -0.31263249, -0.35271661, ..., -0.48313587,
         0.33306149,  0.        ],
       [ 0.73866846,  0.49856417,  1.21173089, ...,  0.91530395,
         1.0845286 ,  0.        ],
       ...,
       [-0.64338415, -0.71823082, -0.65070661, ..., -0.68878878,
        -0.04267206,  0.        ],
       [-0.02639638, -0.31263249,  0.09426839, ...,  0.02414132,
         0.70879505,  0.        ],
       [-0.48708059, -0.58303138, -0.35271661, ..., -0.45571548,
         0.70879505,  0.        ]], shape=(100000, 20))

In [57]:
kmeans = KMeans(n_clusters=20, random_state=1)
kmeans.fit(scaled_X)
features_df = features_df.with_columns(pl.Series("cluster_id", kmeans.labels_)); features_df

In [58]:
features_df.sink_parquet("/home/muthu/Desktop/New Folder/notebook/features_clustered.parquet")
#features_df.to_pandas().to_csv("/home/muthu/Desktop/New Folder/notebook/features_clustered.csv", index=False)

In [59]:
cluster_summary = (
    features_df.group_by("cluster_id")
    .agg([
        pl.count().alias("cluster_size"),
        pl.mean("event_count").alias("avg_events"),
        pl.mean("purchase_count").alias("avg_purchases"),
    ])
); cluster_summary


/tmp/ipykernel_24949/3918410714.py:4: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("cluster_size"),


In [60]:
cluster_summary = cluster_summary.with_columns(
    ("cluster_id " + pl.col("cluster_id").cast(str) + ": " +
     pl.col("cluster_size").cast(str) + " users, " +
     pl.col("avg_events").round(1).cast(str) + " avg events").alias("summary_text")
); cluster_summary.collect()


cluster_id,cluster_size,avg_events,avg_purchases,summary_text
i32,u32,f64,f64,str
13,1848,125.040043,6.026515,"""cluster_id 13: 1848 users, 125…"
14,2175,297.523218,17.037241,"""cluster_id 14: 2175 users, 297…"
5,15025,2.522529,0.034476,"""cluster_id 5: 15025 users, 2.5…"
15,7097,138.887135,7.877131,"""cluster_id 15: 7097 users, 138…"
2,6001,137.267289,7.707715,"""cluster_id 2: 6001 users, 137.…"
…,…,…,…,…
3,1829,283.332969,16.323127,"""cluster_id 3: 1829 users, 283.…"
0,24330,72.577435,3.997287,"""cluster_id 0: 24330 users, 72.…"
7,2690,292.359851,16.79777,"""cluster_id 7: 2690 users, 292.…"


In [61]:
cluster_summary.sink_parquet("/home/muthu/Desktop/New Folder/notebook/cluster_summary.parquet")
#cluster_summary.to_pandas().to_csv("/home/muthu/Desktop/New Folder/notebook/cluster_summary.csv", index=False)